# Statistical Comparison of Three ML Techniques

**Project:** Multilabel Classification of Seismic Signals  
**Models compared:**
1. Random Forest (`rf_repkfold.pkl`)
2. Support‑Vector Machine (`svm_repkfold.pkl`)
3. Neural Network (`best_seismic.ckpt` + `scaler.pkl`)

We evaluate the models with **Repeated K‑Fold Cross‑Validation** and statistically compare their F1‑micro performance using the Friedman test and Wilcoxon signed‑rank post‑hoc tests. Adjust the number of repeats/folds if training time is excessive (default `n_repeats=2`, `k=5`).

In [41]:
import os, json, joblib, torch, numpy as np, pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from scipy.stats import friedmanchisquare, wilcoxon
import matplotlib.pyplot as plt
import seaborn as sns  # only for nicer plots, uninstall if not present


In [42]:
if os.path.basename(os.getcwd()) != 'Seismic-Multilabel-Event-Classifier':
    while os.path.basename(os.getcwd()) != 'Seismic-Multilabel-Event-Classifier':
        os.chdir('..')
    print('CWD cambiado →', os.getcwd())

In [43]:

# PyTorch‑Lightning model class must be importable; adjust path if needed
from src.seismic_model import SeismicMultilabelModel  # Adjusted path based on project structure


In [44]:
# --- Load processed JSON dataset ---
path_json = 'data/processed/dataset_final.json'
with open(path_json, 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

label_cols = [
    '1 Stiker Slip (SS)', '2 Normal-Oblique (SO)', '3 Reverse-Oblique (RO)',
    '4-6', '6-8', '0-200', '200-400', '400-600', '600-'
]
feature_cols = [c for c in df.columns if c not in label_cols]

X = df[feature_cols].to_numpy(dtype='float32')
y = df[label_cols].astype(int).to_numpy()
print('Dataset:', X.shape, y.shape)

Dataset: (1674, 82) (1674, 9)


In [47]:
import torch

# 1. Carga el checkpoint “a mano”
ckpt = torch.load('models/best_seismic.ckpt', map_location='cpu')

# 2. Extrae el state_dict y renombra classifier. → model.
old_sd = ckpt['state_dict']
new_sd = {k.replace('classifier.', 'model.'): v for k, v in old_sd.items()}

# 3. Lee los hyperparams guardados
hparams = ckpt['hyper_parameters']
#    hidden_units, lambda_L2, lr, input_shape, num_classes, etc.

# 4. Instancia tu clase con los mismos hyperparams
model = SeismicMultilabelModel(
    input_dim= 82,
    input_shape=hparams['input_shape'],
    num_classes=hparams['num_classes'],
    hidden_units=hparams['hidden_units'],
    lambda_L2=hparams['lambda_L2'],
    lr=hparams['lr']
)

# 5. Carga el estado ya renombrado
model.load_state_dict(new_sd)

RuntimeError: Error(s) in loading state_dict for SeismicMultilabelModel:
	Missing key(s) in state_dict: "model.0.weight", "model.0.bias". 
	Unexpected key(s) in state_dict: "model.5.weight", "model.5.bias", "model.5.running_mean", "model.5.running_var", "model.5.num_batches_tracked", "model.7.weight", "model.7.bias", "model.1.weight", "model.1.bias", "model.2.running_mean", "model.2.running_var", "model.2.num_batches_tracked". 
	size mismatch for model.2.weight: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64, 128]).
	size mismatch for model.2.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for model.4.weight: copying a param with shape torch.Size([64, 128]) from checkpoint, the shape in current model is torch.Size([9, 64]).
	size mismatch for model.4.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is

In [45]:
# --- Helper: build a pipeline with saved hyperparameters ---
# Load pre‑trained scikit‑learn models (they already contain optimal hyperparameters)
rf_template = joblib.load('models/rf_repkfold.pkl')
svm_template = joblib.load('models/svm_repkfold.pkl')

# clone() makes a fresh estimator with same params, ensuring a new fit each fold
from sklearn.base import clone

# For RF and SVM we wrap in Pipeline to avoid data leakage (fit scaler inside CV)
def make_model(estimator):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('clf', clone(estimator))
    ])

rf_pipe  = make_model(rf_template)
svm_pipe = make_model(svm_template)

# --- Neural Net pipeline (Lightning) ---
scaler_pkl = joblib.load('models/scaler.pkl')  # only for inference; we'll fit another scaler per fold

def train_nn(X_train, y_train, X_val, y_val, hparams, epochs=20):
    import lightning as L
    from torch.utils.data import TensorDataset, DataLoader
    Xsc = StandardScaler().fit_transform(X_train)
    Xv  = StandardScaler().fit_transform(X_val)
    Xtr_tensor = torch.tensor(Xsc, dtype=torch.float32)
    ytr_tensor = torch.tensor(y_train, dtype=torch.float32)
    Xv_tensor  = torch.tensor(Xv, dtype=torch.float32)
    yv_tensor  = torch.tensor(y_val, dtype=torch.float32)
    train_ds   = TensorDataset(Xtr_tensor, ytr_tensor)
    val_ds     = TensorDataset(Xv_tensor,  yv_tensor)
    train_dl   = DataLoader(train_ds, batch_size=32, shuffle=True)
    val_dl     = DataLoader(val_ds,   batch_size=32, shuffle=False)
    model = SeismicMultilabelModel(
        input_shape=X_train.shape[1],
        num_classes=y_train.shape[1],
        hidden_units=hparams['hidden_units'],
        lambda_L2=hparams['lambda_L2'],
        lr=hparams['lr']
    )
    trainer = L.Trainer(max_epochs=epochs, logger=False, enable_progress_bar=False, enable_checkpointing=False,
                        accelerator='gpu' if torch.cuda.is_available() else 'cpu', devices=1)
    trainer.fit(model, train_dl, val_dl)
    return model, StandardScaler().fit(X_train)  # return fresh scaler too

# Hyperparameters from best_seismic.ckpt (stored in checkpoint)
ckpt_path = 'models/best_seismic.ckpt'

best_nn = SeismicMultilabelModel.load_from_checkpoint(
    ckpt_path,
    input_dim=82,       # ← n-features en X
    num_classes=9       # ← n-etiquetas
)
hparams = {
    'hidden_units': best_nn.hparams.hidden_units,
    'lambda_L2':    best_nn.hparams.lambda_L2,
    'lr':           best_nn.hparams.lr
}


RuntimeError: Error(s) in loading state_dict for SeismicMultilabelModel:
	Missing key(s) in state_dict: "model.0.weight", "model.0.bias", "model.2.weight", "model.2.bias", "model.4.weight", "model.4.bias". 
	Unexpected key(s) in state_dict: "classifier.1.weight", "classifier.1.bias", "classifier.2.weight", "classifier.2.bias", "classifier.2.running_mean", "classifier.2.running_var", "classifier.2.num_batches_tracked", "classifier.4.weight", "classifier.4.bias", "classifier.5.weight", "classifier.5.bias", "classifier.5.running_mean", "classifier.5.running_var", "classifier.5.num_batches_tracked", "classifier.7.weight", "classifier.7.bias". 

In [ ]:
# ---- Repeated K‑Fold evaluation ----
n_splits = 5
n_repeats = 2
rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

rf_scores, svm_scores, nn_scores = [], [], []

for fold, (train_idx, val_idx) in enumerate(rkf.split(X)):
    print(f'Fold {fold+1}/{n_splits*n_repeats}')
    X_train, y_train = X[train_idx], y[train_idx]
    X_val,   y_val   = X[val_idx],   y[val_idx]

    # --- Random Forest ---
    rf = clone(rf_pipe)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rf_scores.append(f1_score(y_val, y_pred, average='micro'))

    # --- SVM ---
    svm = clone(svm_pipe)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_val)
    svm_scores.append(f1_score(y_val, y_pred, average='micro'))

    # --- Neural Net --- (lightweight epochs=15)
    nn_model, scaler_fold = train_nn(X_train, y_train, X_val, y_val, hparams, epochs=15)
    X_val_scaled = scaler_fold.transform(X_val)
    with torch.no_grad():
        logits = nn_model(torch.tensor(X_val_scaled, dtype=torch.float32))
        probs  = torch.sigmoid(logits).numpy()
    y_pred = (probs > 0.5).astype(int)
    nn_scores.append(f1_score(y_val, y_pred, average='micro'))

# Store results in DataFrame
df_scores = pd.DataFrame({
    'RandomForest': rf_scores,
    'SVM':          svm_scores,
    'NeuralNet':    nn_scores
})
df_scores.head()

In [ ]:
print(df_scores.describe())

plt.figure(figsize=(8,5))
sns.boxplot(data=df_scores)
plt.ylabel('F1‑micro')
plt.title('Repeated K‑Fold distribution (n_repeats=2, k=5)')
plt.show()

In [ ]:
# --- Friedman test (overall) ---
stat, p = friedmanchisquare(df_scores['RandomForest'], df_scores['SVM'], df_scores['NeuralNet'])
print(f'Friedman χ² = {stat:.4f}, p‑value = {p:.4e}')

alpha = 0.05
if p < alpha:
    print('» Significant differences among models -> running Wilcoxon pairwise tests')
    pairs = [('RandomForest','SVM'), ('RandomForest','NeuralNet'), ('SVM','NeuralNet')]
    for a,b in pairs:
        w_stat, p_pair = wilcoxon(df_scores[a], df_scores[b])
        print(f'  {a} vs {b}: W={w_stat:.3f}, p={p_pair:.4e}')
else:
    print('» No statistically significant difference detected (fail to reject H0)')

In [ ]:
df_scores.to_csv('models/repeated_kfold_scores.csv', index=False)
print('Saved per‑fold scores to models/repeated_kfold_scores.csv')